<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/UTXO_data_analysis_Task_2_with_extended_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime

In [ ]:
!pip install chart_studio

     |████████████████████████████████| 71kB 3.4MB/s 


## Import Data from Google Drive and Data Wrangling

In [ ]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

# Mounting drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
df_2010=pd.read_csv('/content/drive/My Drive/joint_2010.csv')
df_2010.head()

,num,value,block_date,spent_block_date
0,21553,5.000000e+09,1/9/2009,1/12/2009
1,1,5.000000e+09,1/9/2009,NaN
2,2,5.000000e+09,1/9/2009,NaN
3,3,5.000000e+09,1/9/2009,NaN
4,4,5.000000e+09,1/9/2009,NaN


In [ ]:
df_2011=pd.read_csv('/content/drive/My Drive/joint_2011.csv')
df_2011.tail()

,num,value,block_date,spent_block_date
1048570,415429,8.496683e+06,8/6/2011,8/8/2011
1048571,415965,1.200000e+08,8/6/2011,8/8/2011
1048572,416087,1.700000e+07,8/6/2011,8/8/2011
1048573,416508,5.884410e+07,8/6/2011,8/8/2011
1048574,420271,1.222000e+09,8/6/2011,8/8/2011


In [ ]:
merge = [df_2010, df_2011]
df = pd.concat(merge)
df.head()

,num,value,block_date,spent_block_date
0,21553,5.000000e+09,1/9/2009,1/12/2009
1,1,5.000000e+09,1/9/2009,NaN
2,2,5.000000e+09,1/9/2009,NaN
3,3,5.000000e+09,1/9/2009,NaN
4,4,5.000000e+09,1/9/2009,NaN


In [ ]:
# Generate the UTXO value in bitcoin unit, which = $value/10^{8}$
df['UTXO'] = df['value']*10**(-8)
df = df.reset_index()
df = df.drop(['value','index','num'], axis = 1)
df.tail()

,block_date,spent_block_date,UTXO
1186094,8/6/2011,8/8/2011,0.084967
1186095,8/6/2011,8/8/2011,1.200000
1186096,8/6/2011,8/8/2011,0.170000
1186097,8/6/2011,8/8/2011,0.588441
1186098,8/6/2011,8/8/2011,12.220000


In [ ]:
# Change the block_date and spent_block_date to datatime object
df['block_date'] = pd.to_datetime(df['block_date'], format='%m/%d/%Y')
df['spent_block_date'] = pd.to_datetime(df['spent_block_date'], format='%m/%d/%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186099 entries, 0 to 1186098
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   block_date        1186099 non-null  datetime64[ns]
 1   spent_block_date  1164547 non-null  datetime64[ns]
 2   UTXO              1186099 non-null  float64       
dtypes: datetime64[ns](2), float64(1)
memory usage: 27.1 MB


#Task 2: The Weighted Average of Life Expectancy by UTXO Value

We calculate the weighted average life expectancy for all UTXO that has been spent by each date during 2009-01-09 to 2010-12-31

*   The weights are UTXO VALUE



In [ ]:
from datetime import datetime
df['Life_Expectancy'] =df['spent_block_date']-df['block_date']
df.tail()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE
32775,2009-12-31,NaT,50.0,NaT,NaN
32776,2009-12-31,NaT,50.0,NaT,NaN
32777,2009-12-31,NaT,50.0,NaT,NaN
32778,2009-12-31,NaT,50.0,NaT,NaN
32779,2009-12-31,NaT,50.0,NaT,NaN


In [ ]:
### only reserve the value of days
df['Life_Expectancy']=df['Life_Expectancy'].map(lambda x:x.days)
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE
0,2009-01-09,2009-01-12,50.0,3.0,3.000000
168,2009-01-12,2009-01-12,29.0,0.0,1.898734
169,2009-01-12,2009-01-12,1.0,0.0,1.875000
170,2009-01-12,2009-01-12,1.0,0.0,1.851852
171,2009-01-12,2009-01-12,40.0,0.0,1.239669


In [ ]:
df = df.sort_values(by = 'spent_block_date')

In [ ]:
#https://stackoverflow.com/questions/49573844/pandas-cumulative-weighted-average
df['WALE']=(df['UTXO'].mul(df['Life_Expectancy'])).cumsum().div(df['UTXO'].cumsum())
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE
0,2009-01-09,2009-01-12,50.0,3.0,3.000000
168,2009-01-12,2009-01-12,29.0,0.0,1.898734
169,2009-01-12,2009-01-12,1.0,0.0,1.875000
170,2009-01-12,2009-01-12,1.0,0.0,1.851852
171,2009-01-12,2009-01-12,40.0,0.0,1.239669


In [ ]:
### for each date, we count until the last UTXO spent
df_WALE=df.groupby('spent_block_date')['WALE'].max().reset_index()
df_WALE.head()

,spent_block_date,WALE
0,2009-01-12,3.000000
1,2009-01-14,0.925000
2,2009-01-15,1.413636
3,2009-01-16,1.353191
4,2009-01-18,1.442202


In [ ]:
df_newborn = pd.read_csv('/content/drive/My Drive/df_newborn.csv', index_col = 0)
df_newborn['block_date'] = pd.to_datetime(df_newborn['block_date'], format='%Y/%m/%d')
df_newborn.tail()

,block_date,UTXO_newborn
935,2011-08-02,85296.723708
936,2011-08-03,113896.630558
937,2011-08-04,88667.639105
938,2011-08-05,147648.110740
939,2011-08-06,77115.643571


In [ ]:
df_dead = pd.read_csv('/content/drive/My Drive/df_dead.csv',index_col = 0)
df_dead['spent_block_date'] = pd.to_datetime(df_dead['spent_block_date'], format='%Y/%m/%d')
df_dead.tail()

,spent_block_date,UTXO_dead
3791,2020-10-11,1050.000000
3792,2020-10-14,64.859446
3793,2020-10-15,0.487548
3794,2020-10-16,5.510000
3795,2020-10-17,50.000000


In [ ]:
np.size(pd.date_range(start='2009-01-09', end='2011-08-06'))

940

In [ ]:
df_UTXO = pd.DataFrame(np.zeros((940, 1)))
df_UTXO.columns=['date']
df_UTXO['date']=pd.date_range(start='2009-01-09', end='2011-08-06')
df_UTXO.tail()

,date
935,2011-08-02
936,2011-08-03
937,2011-08-04
938,2011-08-05
939,2011-08-06


In [ ]:
# merge
df_UTXO=df_UTXO.merge(df_newborn, how='left',left_on='date',right_on='block_date')

df_UTXO=df_UTXO.merge(df_dead, how='left',left_on='date',right_on='spent_block_date')
df_UTXO = df_UTXO.drop(['block_date','spent_block_date'], axis = 1)
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead
0,2009-01-09,700.0,NaN
1,2009-01-10,3050.0,NaN
2,2009-01-11,4650.0,NaN
3,2009-01-12,4879.0,179.0
4,2009-01-13,6150.0,NaN


In [ ]:
df_UTXO=df_UTXO.merge(df_WALE, how='left',left_on='date',right_on='spent_block_date')
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead,spent_block_date,WALE
0,2009-01-09,700.0,NaN,NaT,NaN
1,2009-01-10,3050.0,NaN,NaT,NaN
2,2009-01-11,4650.0,NaN,NaT,NaN
3,2009-01-12,4879.0,179.0,2009-01-12,3.0
4,2009-01-13,6150.0,NaN,NaT,NaN


In [ ]:
df_UTXO.to_csv('/content/drive/My Drive/df_UTXO.csv')

In [ ]:
df_UTXO2=df_UTXO.drop('spent_block_date',axis=1)
df_UTXO2=df_UTXO.fillna(method='ffill')
df_UTXO2=df_UTXO.dropna()
df_UTXO2.head()

,date,UTXO_newborn,UTXO_dead,spent_block_date,WALE
3,2009-01-12,4879.0,179.0,2009-01-12,3.000000
5,2009-01-14,6511.0,61.0,2009-01-14,0.925000
6,2009-01-15,6800.0,500.0,2009-01-15,1.413636
7,2009-01-16,5600.0,200.0,2009-01-16,1.353191
9,2009-01-18,5500.0,150.0,2009-01-18,1.442202


In [ ]:
import plotly.express as px
from plotly.offline import iplot

fig2 = px.line(df_UTXO2, x = 'date', y = 'WALE', title = 'The Weighted Average of Life Expectancy by UTXO Value')

# Add range slider
fig2.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

iplot(fig2)